In [57]:
!pip install transformers keybert sentence-transformers --quiet

In [58]:
from transformers import pipeline
from keybert import KeyBERT
import re


In [59]:
def clean_text(text: str) -> str:
    text = re.sub(r'\s+', ' ', text).strip()
    return text


In [60]:
def summarize_text(text: str) -> str:
    """
    Summarize input text into exactly 3 bullet points.
    If fewer than 3 points are generated, reuse earlier ones for padding.
    """
    import re

    text = clean_text(text)
    word_count = len(text.split())

    if word_count < 30:
        return "Please provide a longer paragraph (minimum 30 words) for an effective summary."

    try:
        max_len = max(40, min(130, int(word_count * 0.6)))
        min_len = max(20, min(30, int(word_count * 0.3)))
        if min_len >= max_len:
            min_len = max_len - 5

        summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6", device=-1)
        summary = summarizer(text, max_length=max_len, min_length=min_len, do_sample=False)
        result = summary[0]['summary_text']

        # Split into sentences
        sentences = re.split(r'(?<=[.!?])\s+', result)
        sentences = [s.strip() for s in sentences if s.strip()]

        # Pad with repeated sentences if fewer than 3
        while len(sentences) < 3:
            sentences.append(sentences[len(sentences) % len(sentences)])  # repeat from start

        # Format as bullet points
        bullet_points = [f"• {sentence}" for sentence in sentences[:3]]
        return "\n".join(bullet_points)

    except Exception as e:
        return f"An error occurred while summarizing: {str(e)}"


In [61]:
def extract_keywords(text: str, top_n: int = 5) -> list:
    text = clean_text(text)
    if not text or len(text.split()) < 10:
        return ["Please provide a longer input (minimum 10 words) to extract meaningful keywords."]

    try:
        kw_model = KeyBERT()
        keywords = kw_model.extract_keywords(text, top_n=top_n)
        return [kw for kw, _ in keywords]
    except Exception as e:
        return [f"An error occurred while extracting keywords: {str(e)}"]


In [62]:
sample_text = """
Artificial Intelligence (AI) is reshaping modern enterprises by automating routine workflows,
extracting deep insights from big data, and personalizing customer interactions. In education,
AI helps tailor content delivery to individual learning styles. At Senthuron, tools like these
can streamline client deliverables, enhance internal decision-making, and power new digital
experiences.
"""


In [63]:
# Summarize
print("3-Line Summary:\n")
print(summarize_text(sample_text))

# Extract Keywords
print("\nTop 5 Keywords:\n")
for i, kw in enumerate(extract_keywords(sample_text), 1):
    print(f"{i}. {kw}")


3-Line Summary:



Device set to use cpu


• Artificial Intelligence (AI) is reshaping modern enterprises by automating routine workflows and extracting deep insights from big data .
• In education, AI helps tailor content delivery to individual learning styles .
• Artificial Intelligence (AI) is reshaping modern enterprises by automating routine workflows and extracting deep insights from big data .

Top 5 Keywords:

1. ai
2. intelligence
3. learning
4. automating
5. education
